<a href="https://colab.research.google.com/github/nikolmash/legal_summarization/blob/main/semantic_shifts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обнаружение семантических сдвигов между юридическими и новостными текстами

Загрузка судебных данных:
(в данной задаче используется небольшая тестовая выборка в целях экономии затрат на память и вычислимость)

In [ ]:
!wget https://www.dropbox.com/s/mbj3sb6jaw3d9s3/judgements_test.json

--2021-06-02 18:57:35--  https://www.dropbox.com/s/mbj3sb6jaw3d9s3/judgements_test.json
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/mbj3sb6jaw3d9s3/judgements_test.json [following]
--2021-06-02 18:57:36--  https://www.dropbox.com/s/raw/mbj3sb6jaw3d9s3/judgements_test.json
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd5c518edbb6afa63765643ac95.dl.dropboxusercontent.com/cd/0/inline/BPqRWBK5wxshR-XGPkfb0Ns7KLtw5-8Tpxi57IcT-waUZkAiYevM-p8KVoZ8kfkufyXfLu5F1zKbMQjBMsJ_7sF0Nr4oQdWsTbHhOSfAuMPcu6r0h3F8d4BObHdi8aDqD9xF2xAJ6DbbVmftW9DnO_w-/file# [following]
--2021-06-02 18:57:36--  https://ucd5c518edbb6afa63765643ac95.dl.dropboxusercontent.com/cd/0/inline/BPqRWBK5wxshR-XGPkfb0Ns7KLtw5-8Tpxi57IcT-waUZkAiYevM-p8KVoZ8kfk

In [ ]:
with open ('judgements_test.json', encoding='utf-8') as f:
  data_judge = json.load(f)

Установка библиотек для лемматизации и токенизации (это будет необходимо для идентификации интересующих нас предложений)

In [2]:
!pip install pymorphy2 git+https://github.com/Koziev/rutokenizer

  Cloning https://github.com/Koziev/rutokenizer to /tmp/pip-req-build-u_30b7c9
  Running command git clone -q https://github.com/Koziev/rutokenizer /tmp/pip-req-build-u_30b7c9
     |████████████████████████████████| 61kB 3.4MB/s 
     |████████████████████████████████| 8.2MB 7.1MB/s 
  Created wheel for rutokenizer: filename=rutokenizer-0.0.24-cp37-none-any.whl size=30370096 sha256=ae3a7286afee9c7f1850227bc9d5f079c6418affbe3b2da91bc88f8ca17a13a6
  Stored in directory: /tmp/pip-ephem-wheel-cache-0zk_6w5r/wheels/a6/66/ef/7c52e143b099c8aeaaf3bccc4640c065b87d7cbfb13066005d
Successfully built rutokenizer


Импорт основных необходимых пакетов

In [3]:
import json
import rutokenizer
import pymorphy2
import numpy as np
import pandas as pd
from collections import Counter
from tqdm.auto import tqdm
from string import punctuation

Создание основных объектов и сущностей, а также функция для предобработки: токенизации, удаления запятых и лемматизации

In [ ]:
punkt = punctuation + '«»—…“”*№–'
morph = pymorphy2.MorphAnalyzer()
t = rutokenizer.Tokenizer()
t.load()

def lemmatizing(text):
  tokens = t.tokenize(text)
  lemmas = [morph.parse(word)[0].normal_form for word in tokens if word not in punkt]
  lemmas = [w.lower() for w in lemmas]
  return lemmas

In [ ]:
vocabulary = Counter()


for ind in tqdm(data_judge.keys()):
  
  lemmas = lemmatizing(data_judge[ind]['text'])
  vocabulary.update(lemmas)

Так как невозможно будет сравнить векторные представления всех пересекающихся слов в корпусе юридических и новостных текстов, то необходимо выбрать какое-то конечно множество слов, для которых будет применяться метод. Очевидно, что нас интересуют характерная судебная лексика, поэтому выберем для этой задачи 1000 наиболее частотных слов из постановлений, причем только существительных, прилагательных и глаголов (иначе - наиболее частотными окажутся, в основном, слова служебных частей речи. именно эти части речи - чтобы ограничить выборку, и, кажется, что смысл таких частей речи привычнее анаизировать, они "основные")

In [ ]:
filtered_vocab = {word:vocabulary[word] for word in vocabulary.keys() if morph.parse(word)[0].tag.POS in ['NOUN', 'ADJF', 'VERB']}
filtered_vocab = Counter(filtered_vocab)

In [ ]:
filtered_words = [el[0] for el in filtered_vocab.most_common(1000)]

В качестве второго корпуса будут использованы новостные статьи Lenta.ru (источник: https://github.com/yutkin/Lenta.Ru-News-Dataset) 

In [ ]:
!wget https://www.dropbox.com/s/v9i2nh12a4deuqj/lenta.tar.gz

--2021-06-01 10:55:48--  https://www.dropbox.com/s/v9i2nh12a4deuqj/lenta.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/v9i2nh12a4deuqj/lenta.tar.gz [following]
--2021-06-01 10:55:49--  https://www.dropbox.com/s/raw/v9i2nh12a4deuqj/lenta.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucefa1c5256d62e48649154415bf.dl.dropboxusercontent.com/cd/0/inline/BPleBtl6np5KP_GpauUccdnxMwIR0jYNRzqvnVoQNy1lVkGlclif1MlyTweBK0Lfe1RCpQk1Jc-KY0w3H7-zXRtBcd6CLAvq_97u8B-jA1XZX1a3fu5czOmaViO_pqDSLyF9HIAt-95YACM9DXmzTFfx/file# [following]
--2021-06-01 10:55:49--  https://ucefa1c5256d62e48649154415bf.dl.dropboxusercontent.com/cd/0/inline/BPleBtl6np5KP_GpauUccdnxMwIR0jYNRzqvnVoQNy1lVkGlclif1MlyTweBK0Lfe1RCpQk1Jc-KY0w3H7-zXRtBcd

Распаковываем архив

In [ ]:
!tar -xf lenta.tar.gz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tar: Ignoring unknown extended header keyword 'SCHILY.dev'
tar: Ignoring unknown extended header keyword 'SCHILY.ino'
tar: Ignoring unknown extended header keyword 'SCHILY.nlink'


Объем датасета очень большой: несколько миллионов статей. Во-первых, нецелесообразно будет сопоставлять данный корпус достаточно маленькому судебному корпусу, во-вторых, опять же необходима экономия вычислительных ресурсов. Поэтому используем набор тестовых данных (интересуют только сами тексты), и обрежем таблицу до 30 тысяч строк

In [ ]:
lenta = pd.read_csv('lenta-ru-news.test.csv', nrows=30000, error_bad_lines=False)
lenta.head()

b'Skipping line 108: expected 5 fields, saw 12\nSkipping line 698: expected 5 fields, saw 20\nSkipping line 812: expected 5 fields, saw 8\nSkipping line 1116: expected 5 fields, saw 10\nSkipping line 1390: expected 5 fields, saw 16\nSkipping line 1531: expected 5 fields, saw 24\nSkipping line 1677: expected 5 fields, saw 6\nSkipping line 1796: expected 5 fields, saw 15\nSkipping line 2982: expected 5 fields, saw 33\nSkipping line 3301: expected 5 fields, saw 19\nSkipping line 3415: expected 5 fields, saw 12\nSkipping line 3456: expected 5 fields, saw 13\nSkipping line 3567: expected 5 fields, saw 18\nSkipping line 3646: expected 5 fields, saw 7\nSkipping line 3717: expected 5 fields, saw 7\nSkipping line 3744: expected 5 fields, saw 17\nSkipping line 3873: expected 5 fields, saw 16\nSkipping line 4087: expected 5 fields, saw 6\nSkipping line 4144: expected 5 fields, saw 15\nSkipping line 4355: expected 5 fields, saw 13\nSkipping line 4466: expected 5 fields, saw 12\nSkipping line 5048:

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/15/ovechkin/,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт,Хоккей
1,https://lenta.ru/news/2018/12/15/gaz/,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика,Госэкономика
2,https://lenta.ru/news/2018/12/15/kandidat/,Главой украинских раскольников захотели сделат...,Предстоятелем новой украинской православной це...,Бывший СССР,Украина
3,https://lenta.ru/news/2018/12/15/putinrap/,Путин предостерег от запретов рэп-концертов,"Президент России Владимир Путин, выступая на з...",Культура,Музыка
4,https://lenta.ru/news/2018/12/15/paris/,В Париже на акции протеста арестовали 100 человек,В Париже арестован 101 человек во время акции ...,Мир,Происшествия


Удалются пропущенные значения:

In [ ]:
lenta = lenta.dropna()
lenta.shape

(26903, 5)

Применяем функцию предобработки ко всем новостным текстам: поскольку нужно будет искать предложения, где содержатся топ-1000 частотных "судебных" интересующих нас слов, а они хранятся в начальной форме

In [ ]:
tqdm.pandas()

lenta['tokenized'] = lenta['text'].progress_apply(lambda x: lemmatizing(x))

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Сохраним в список индексы тех предложений, в которых содержатся эти топ-1000 слов

In [ ]:
ind_news = []

for w in tqdm(filtered_words):
  for n_i in range(lenta.shape[0]):
    if w in lenta.iloc[n_i]['tokenized']:
      if n_i not in ind_news:
        ind_news.append(n_i)

И отдельно будем хранить эти предложения:

In [ ]:
sentences_news = []

for ind in ind_news:
  tokens = [word for word in t.tokenize(lenta.iloc[int(ind)]['text']) if word not in punkt]
  sentences_news.append(tokens)

Устанавливаем библиотеку для удобной работы с моделью ELMO (https://github.com/ltgoslo/simple_elmo)

In [ ]:
!pip install --upgrade simple_elmo

     |████████████████████████████████| 51kB 4.0MB/s 


С сайта rusvectores.org загружаем контекстуализированную модель, обученную на корпусе Тайга, объем которого - почти 5 миллиардов слов

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/199.zip

--2021-06-02 18:58:56--  http://vectors.nlpl.eu/repository/20/199.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1800442066 (1.7G) [application/zip]
Saving to: ‘199.zip’

199.zip             100%[===================>]   1.68G  5.54MB/s    in 4m 34s  

2021-06-02 19:03:31 (6.27 MB/s) - ‘199.zip’ saved [1800442066/1800442066]



Подгружаем эту модель в simple_elmo

In [ ]:
from simple_elmo import ElmoModel

model = ElmoModel()
model.load('199.zip')

2021-06-02 19:03:33,020 : INFO : Loading model from 199.zip...
2021-06-02 19:03:33,022 : INFO : 
            Assuming the model is a ZIP archive downloaded from the NLPL vector repository.
            Loading a model from a ZIP archive directly is slower than from the extracted files,
            but does not require additional disk space
            and allows to load from directories without write permissions.
            
2021-06-02 19:03:33,026 : INFO : We will cache the vocabulary of 100 tokens.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_varia

'The model is now loaded.'

Теперь нужно для всех слов, используя те предложений из новостных статей, где содержатся они содержатся, получить контекстуализированный эмбеддинм. Для хранения используем словарь, где ключи - судебные слова, а значения - один или несколько векторов, размерности 1024. По умолчанию, функия get_elmo_vectors возвращает среднее всех скрытых представлений из модели ELMO. 


Ниже приведенный цикл работает достаточно долго, поэтому объем предложений, для которых будут получаться эмбеддинги был обрезан

In [7]:
from collections import OrderedDict

In [ ]:
token_embeddings_news = OrderedDict()



for i in tqdm(range(0,3000,20)):
  slic = sentences_news[i:i+20]
  embs = model.get_elmo_vectors(slic)
  for w in filtered_words:
    for s_ind in range(len(slic)):
      if w in slic[s_ind]:
        token_indexes = [i for i,val in enumerate(slic[s_ind]) if val==w]
        if w in token_embeddings_news.keys():
          token_embeddings_news[w] = np.vstack((token_embeddings_news[w], 
                                                embs[s_ind][token_indexes]))
        else:
          token_embeddings_news[w] = embs[s_ind][token_indexes]


2021-06-01 11:06:32,668 : INFO : Warming up ELMo on 20 sentences...
2021-06-01 11:07:25,087 : INFO : Warming up finished.
2021-06-01 11:07:25,117 : INFO : Texts in the current batch: 20
2021-06-01 11:08:15,359 : INFO : Warming up ELMo on 20 sentences...
2021-06-01 11:09:32,274 : INFO : Warming up finished.
2021-06-01 11:09:32,310 : INFO : Texts in the current batch: 20
2021-06-01 11:10:49,166 : INFO : Warming up ELMo on 20 sentences...
2021-06-01 11:11:56,114 : INFO : Warming up finished.
2021-06-01 11:11:56,141 : INFO : Texts in the current batch: 20
2021-06-01 11:13:02,502 : INFO : Warming up ELMo on 20 sentences...
2021-06-01 11:14:02,366 : INFO : Warming up finished.
2021-06-01 11:14:02,405 : INFO : Texts in the current batch: 20
2021-06-01 11:15:03,280 : INFO : Warming up ELMo on 20 sentences...
2021-06-01 11:16:15,786 : INFO : Warming up finished.
2021-06-01 11:16:15,816 : INFO : Texts in the current batch: 20
2021-06-01 11:17:27,316 : INFO : Warming up ELMo on 20 sentences...
20

Для скольки слов были получены векторные представления:

In [ ]:
len(token_embeddings_news)

739

Далее нужно то же самое проделать с текстами судебных постановлений. Сначала отдельно также извлечем и сохраним предложения, которые содержат топ-1000 частотных слов. Важно здесь отметить, что модель может получать на вход максимум 2048 токенов - поэтому каждый текст, длиннее этого значения, был обрезан

In [ ]:
sentences_judg = []

for ind in data_judge.keys():
  j_tokens = [word for word in t.tokenize(data_judge[ind]['text']) if word not in punkt][:2048]
  sentences_judg.append(j_tokens)

Цикл с получением эмбеддинга для каждого слова представлен ниже. Так как юридические тексты намного более длинные, чем новостные, работа с ними занимает очень большое количество времени, и даже объем тестовой выборки (2680 текстов) обрабатывался бы более суток , поэтому работа ячейки была приостановлена

In [ ]:
token_embeddings_judg = OrderedDict()

for i in tqdm(range(0,2680,10)):
  slic = sentences_judg[i:i+10]
  embs = model.get_elmo_vectors(slic)
  for w in filtered_words:
    for j_ind in range(len(slic)):
      if w in slic[j_ind]:
        token_indexes = [i for i,val in enumerate(slic[j_ind]) if val==w]
        if w in token_embeddings_judg.keys():
          token_embeddings_judg[w] = np.vstack((token_embeddings_judg[w], 
                                           embs[j_ind][token_indexes]))
        else:
          token_embeddings_judg[w] = embs[j_ind][token_indexes]

2021-06-02 19:09:53,521 : INFO : Warming up ELMo on 10 sentences...
2021-06-02 19:12:36,442 : INFO : Warming up finished.
2021-06-02 19:12:36,535 : INFO : Texts in the current batch: 10
2021-06-02 19:15:16,680 : INFO : Warming up ELMo on 10 sentences...
2021-06-02 19:18:35,489 : INFO : Warming up finished.
2021-06-02 19:18:35,570 : INFO : Texts in the current batch: 10
2021-06-02 19:21:43,006 : INFO : Warming up ELMo on 10 sentences...
2021-06-02 19:25:01,613 : INFO : Warming up finished.
2021-06-02 19:25:01,694 : INFO : Texts in the current batch: 10
2021-06-02 19:28:22,130 : INFO : Warming up ELMo on 10 sentences...
2021-06-02 19:31:39,089 : INFO : Warming up finished.
2021-06-02 19:31:39,184 : INFO : Texts in the current batch: 10
2021-06-02 19:34:59,089 : INFO : Warming up ELMo on 10 sentences...
2021-06-02 19:38:18,515 : INFO : Warming up finished.
2021-06-02 19:38:18,600 : INFO : Texts in the current batch: 10
2021-06-02 19:41:26,892 : INFO : Warming up ELMo on 10 sentences...
20

KeyboardInterrupt: ignored

Для скольки слов получилось извлечь эмбеддинги:

In [ ]:
len(token_embeddings_judg)

780

Ниже закоментированные ячейки загружают временные файлы , которые я сохраняла, чтобы не потерять прогресс

In [15]:
"""n_matrix = np.load('drive/MyDrive/tokens_matrix.npy')
j_matrix = np.load('drive/MyDrive/jtokens_matrix.npy')"""

In [28]:
"""token_embeddings_news = OrderedDict()

tokens_news_keys = list(tokens_news.keys())
tokens_news_ind = np.cumsum(list(tokens_news.values()))
for ind in range(len(tokens_news)):
  if ind==0:
    end_ind = tokens_news_ind[0]
    token_embeddings_news[tokens_news_keys[0]] = n_matrix[0:end_ind]
  else:
    start_ind = tokens_news_ind[ind-1]
    end_ind = tokens_news_ind[ind]
    token_embeddings_news[tokens_news_keys[ind]] = n_matrix[start_ind:end_ind]"""

In [36]:
"""token_embeddings_judg = OrderedDict()

tokens_judg_keys = list(tokens_judg.keys())
tokens_judg_ind = np.cumsum(list(tokens_judg.values()))

for ind in range(len(tokens_judg)):
  if ind==0:
    end_ind = tokens_judg_ind[0]
    token_embeddings_judg[tokens_judg_keys[0]] = j_matrix[0:end_ind]
  else:
    start_ind = tokens_judg_ind[ind-1]
    end_ind = tokens_judg_ind[ind]
    token_embeddings_judg[tokens_judg_keys[ind]] = j_matrix[start_ind:end_ind]"""


Найдем пересечение тех слов, для которых есть эмбеддинги в обоих корпусах, и которые будем сравнивать

In [47]:
tokens_news_keys = set(token_embeddings_news.keys())
tokens_judg_keys = set(token_embeddings_judg.keys())

In [49]:
words_to_compare = tokens_news_keys.intersection(tokens_judg_keys)
len(words_to_compare)

651

Далее используются два метода для обнаружения семантических сдвигов из Distributional word embeddings in modeling diachronic semantic change Андрей Кутузова

**Inverted cosine similarity over word prototypes (PRT)**

Считаем для каждого слова среднее всех эмбеддингов (прототип)

In [55]:
average_embeddings_news = {w:np.mean(token_embeddings_news[w], axis=0) for w in words_to_compare}
average_embeddings_judg = {w:np.mean(token_embeddings_judg[w], axis=0) for w in words_to_compare}

Для каждого слова получаем значение PRT: единица, деленная на косинусную близость между прототипами двух корпусов

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

prt_dictionary = {}

for word in words_to_compare:
  news_prototype = average_embeddings_news[word]
  judg_prototype = average_embeddings_judg[word]
  prt_score = 1/cosine_similarity(news_prototype.reshape(1, -1),
                                  judg_prototype.reshape(1, -1))[0][0]
  prt_dictionary[word] = prt_score

Отсортированные результаты:

In [67]:
sorted(prt_dictionary.items(), key=lambda item: item[1], reverse=True)

[('энергетический', 1.5894727907394672),
 ('нежилой', 1.4565930317654965),
 ('закрытый', 1.4285684620403716),
 ('существо', 1.4154283825216163),
 ('территориальный', 1.413766050742436),
 ('материальный', 1.408875867843949),
 ('взаимный', 1.3943260127312187),
 ('потребитель', 1.3914760651389269),
 ('реальный', 1.378233993399251),
 ('подобный', 1.3380887809131965),
 ('правильный', 1.3360396086921729),
 ('дефект', 1.3188116728001782),
 ('вещь', 1.3128866226138105),
 ('последующий', 1.3109214962538451),
 ('категория', 1.2962988815914478),
 ('н', 1.2944719854147333),
 ('грузовой', 1.288716731564534),
 ('сетевой', 1.2841589980000674),
 ('природа', 1.280151455182012),
 ('ввод', 1.276630630262071),
 ('иностранный', 1.2758555374971396),
 ('неверный', 1.274822198723781),
 ('открытый', 1.2747292497062903),
 ('строительный', 1.2744559860044737),
 ('п', 1.2661344293760677),
 ('конкуренция', 1.2660677961841398),
 ('этаж', 1.2575188336529957),
 ('должностной', 1.2553824618275082),
 ('информационный',

**Average pairwise cosine distance between token
embeddings (APD)**

В данном методе нужно посчитать для каждого слова косинусное расстояние между всеми возможными парами эмбеддингов разных корпусов, и получить среднее значение

In [69]:
apd_dictionary = {}

for word in words_to_compare:
  news_embs = token_embeddings_news[word]
  judg_embs = token_embeddings_news[word]
  cosine_distances = []
  for i_n in range(news_embs.shape[0]):
    for i_g in range(judg_embs.shape[0]):
      cos_dist = 1 - cosine_similarity(news_embs[i_n].reshape(1, -1),
                                       judg_embs[i_g].reshape(1, -1))
      cosine_distances.append(cos_dist)
  apd_score = 1/(news_embs.shape[0]*judg_embs.shape[0]) * sum(cosine_distances)
  apd_dictionary[word] = apd_score

Отсортированный результат:

In [73]:
sorted(apd_dictionary.items(), key=lambda item: item[1], reverse=True)

[('свой', array([[0.44634737]])),
 ('тот', array([[0.41441234]])),
 ('этот', array([[0.39709165]])),
 ('весь', array([[0.38924156]])),
 ('друг', array([[0.38315611]])),
 ('такой', array([[0.37921777]])),
 ('другой', array([[0.36557771]])),
 ('второй', array([[0.33844819]])),
 ('последний', array([[0.33575439]])),
 ('первый', array([[0.33374651]])),
 ('вид', array([[0.32328725]])),
 ('какой', array([[0.32220191]])),
 ('полный', array([[0.31723607]])),
 ('новый', array([[0.31685888]])),
 ('любой', array([[0.31656474]])),
 ('сам', array([[0.31360703]])),
 ('год', array([[0.30643434]])),
 ('время', array([[0.3038259]])),
 ('один', array([[0.30274808]])),
 ('дом', array([[0.30266307]])),
 ('рабочий', array([[0.2994232]])),
 ('государственный', array([[0.29790825]])),
 ('место', array([[0.29568771]])),
 ('лицо', array([[0.29437377]])),
 ('месяц', array([[0.29145842]])),
 ('день', array([[0.28772572]])),
 ('какой-либо', array([[0.28011467]])),
 ('общий', array([[0.27984897]])),
 ('который', a